# Running a Simple-Step Problem

> **Set up**
>
> To run this notebook, first install the Julia kernel for Jupyter Notebooks using [IJulia](https://julialang.github.io/IJulia.jl/stable/manual/installation/), then [create an environment](https://pkgdocs.julialang.org/v1/environments/) for this tutorial with the packages listed with `using <PackageName>` further down.
>
> This tutorial has demonstrated compatibility with these package versions. If you run into any errors, first check your package versions for consistency using `Pkg.status()`.
>
 > ```
 > Status `~/work/PowerSimulations.jl/PowerSimulations.jl/docs/Project.toml`
 >   [336ed68f] CSV v0.10.16
 >   [a93c6f00] DataFrames v1.8.1
 >   [864edb3b] DataStructures v0.19.3
 >   [e30172f5] Documenter v1.17.0
 >   [d12716ef] DocumenterInterLinks v1.1.0
 >   [35a29f4d] DocumenterTools v0.1.21
 >   [87dc4568] HiGHS v1.21.1
 >   [fc1677e0] HydroPowerSimulations v0.14.0
 >   [2cd47ed4] InfrastructureSystems v3.3.1
 >   [4076af6c] JuMP v1.29.4
 >   [23fbe1c1] Latexify v0.16.10
 >   [98b081ad] Literate v2.21.0
 >   [bed98974] PowerNetworkMatrices v0.18.1
 >   [e690365d] PowerSimulations v0.33.0 `~/work/PowerSimulations.jl/PowerSimulations.jl`
 >   [f00506e0] PowerSystemCaseBuilder v2.2.0
 >   [bcd98974] PowerSystems v5.5.0
 >   [08abe8d2] PrettyTables v3.2.3
 >   [9e3dc215] TimeSeries v0.25.2
 > 
 > ```


## Introduction

`PowerSimulations.jl` supports the construction and solution of optimal power system
scheduling problems (Operations Problems). Operations problems form the fundamental
building blocks for sequential simulations. This example shows how to specify and customize
the mathematics that will be applied to the data with a `ProblemTemplate`,
build and execute a `DecisionModel`, and access the results.

In [ ]:
using PowerSystems
using PowerSimulations
using HydroPowerSimulations
using PowerSystemCaseBuilder
using HiGHS # solver
using Dates

## Data

> *Note*
>
>
> [PowerSystemCaseBuilder.jl](https://github.com/NREL-Sienna/PowerSystemCaseBuilder.jl)
> is a helper library that makes it easier to reproduce examples in the documentation
> and tutorials. Normally you would pass your local files to create the system data
> instead of calling the function `build_system`.
> For more details visit
> [PowerSystemCaseBuilder Documentation](https://nrel-sienna.github.io/PowerSystems.jl/stable/how_to/powersystembuilder/)

In [ ]:
sys = build_system(PSISystems, "modified_RTS_GMLC_DA_sys")

## Define a problem specification with a `ProblemTemplate`

You can create an empty template with:

In [ ]:
template_uc = ProblemTemplate()

Now, you can add a `DeviceModel` for each device type to create an assignment
between PowerSystems device types and the subtypes of `AbstractDeviceFormulation`.
PowerSimulations has a variety of different `AbstractDeviceFormulation` subtypes
that can be applied to different PowerSystems device types, each dispatching to different
methods for populating optimization problem objectives, variables, and constraints.
Documentation on the formulation options for various devices can be found in the
[formulation library docs](https://nrel-sienna.github.io/PowerSimulations.jl/latest/formulation_library/General/#formulation_library)

### Branch Formulations

Here is an example of relatively standard branch formulations. Other formulations allow
for selective enforcement of transmission limits and greater control on transformer settings.

In [ ]:
set_device_model!(template_uc, Line, StaticBranch)
set_device_model!(template_uc, Transformer2W, StaticBranch)
set_device_model!(template_uc, TapTransformer, StaticBranch)

### Injection Device Formulations

Here we define template entries for all devices that inject or withdraw power on the
network. For each device type, we can define a distinct `AbstractDeviceFormulation`. In
this case, we're defining a basic unit commitment model for thermal generators,
curtailable renewable generators, and fixed dispatch (net-load reduction) formulations
for `HydroDispatch` and `RenewableNonDispatch` devices.

In [ ]:
set_device_model!(template_uc, ThermalStandard, ThermalStandardUnitCommitment)
set_device_model!(template_uc, RenewableDispatch, RenewableFullDispatch)
set_device_model!(template_uc, PowerLoad, StaticPowerLoad)
set_device_model!(template_uc, HydroDispatch, HydroDispatchRunOfRiver)
set_device_model!(template_uc, RenewableNonDispatch, FixedOutput)

### Service Formulations

We have two `VariableReserve` types, parameterized by their direction. So, similar to
creating `DeviceModel`s, we can create `ServiceModel`s. The primary difference being
that `DeviceModel` objects define how constraints get created, while `ServiceModel` objects
define how constraints get modified.

In [ ]:
set_service_model!(template_uc, VariableReserve{ReserveUp}, RangeReserve)
set_service_model!(template_uc, VariableReserve{ReserveDown}, RangeReserve)

### Network Formulations

Finally, we can define the transmission network specification that we'd like to model.
For simplicity, we'll choose a copper plate formulation. But there are dozens of
specifications available through an integration with
[PowerModels.jl](https://lanl-ansi.github.io/PowerModels.jl/stable/).

*Note that many formulations will require appropriate data and may be computationally intractable*

In [ ]:
set_network_model!(template_uc, NetworkModel(CopperPlatePowerModel))

## `DecisionModel`

Now that we have a `System` and a `ProblemTemplate`, we can put the two together
to create a `DecisionModel` that we solve.

### Optimizer

It's most convenient to define an optimizer instance upfront and pass it into the
`DecisionModel` constructor. For this example, we can use the free HiGHS solver
with a relatively relaxed MIP gap (`ratioGap`) setting to improve speed.

In [ ]:
solver = optimizer_with_attributes(HiGHS.Optimizer, "mip_rel_gap" => 0.5)

### Build a `DecisionModel`

The construction of a `DecisionModel` essentially applies a `ProblemTemplate`
to `System` data to create a JuMP model.

In [ ]:
problem = DecisionModel(template_uc, sys; optimizer = solver, horizon = Hour(24))
build!(problem; output_dir = mktempdir())

> *Tip*
>
>
> The principal component of the `DecisionModel` is the JuMP model.
> But you can serialize to a file using the following command:
>
> ```julia
> serialize_optimization_model(problem, save_path)
> ```
>
> Keep in mind that if the setting `"store_variable_names"` is set to `False` then
> the file won't show the model's names.

### Solve a `DecisionModel`

In [ ]:
solve!(problem)

## Results Inspection

PowerSimulations collects the `DecisionModel` results into a
`OptimizationProblemResults` struct:

In [ ]:
res = OptimizationProblemResults(problem)

### Optimizer Stats

The optimizer summary is included

In [ ]:
get_optimizer_stats(res)

### Objective Function Value

In [ ]:
get_objective_value(res)

### Variable, Parameter, Auxiliary Variable, Dual, and Expression Values

The solution value data frames for variables, parameters, auxiliary variables, duals, and
expressions can be accessed using the `read_` methods:

In [ ]:
read_variables(res)

Or, you can read a single parameter value for parameters that exist in the results.

In [ ]:
list_parameter_names(res)
read_parameter(res, "ActivePowerTimeSeriesParameter__RenewableDispatch")

## Plotting

Take a look at the plotting capabilities in
[PowerGraphics.jl](https://nrel-sienna.github.io/PowerGraphics.jl/stable/)